In [1]:
import os
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import cv2

# ==============================
# Step 1: Prepare Dataset
# ==============================
DATASET_DIR = r"C:\Users\jk121\Documents\Code\LargeData\Model_for_English_Words"  # 替換為你的數據集路徑
MODEL_PATH = r"C:\Users\jk121\Documents\Code\LargeData\models\keras_model.h5"
LABELS_PATH = r"C:\Users\jk121\Documents\Code\LargeData\models\labels.txt"

# 使用 MediaPipe 進行手部特徵點檢測
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1)

def extract_landmarks(image_path):
    """ 從圖片中提取手部特徵點 (21 個關鍵點) """
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)
    
    if results.multi_hand_landmarks:
        landmarks = []
        for hand_landmarks in results.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                landmarks.append([lm.x, lm.y, lm.z])
        return np.array(landmarks).flatten()  # 平鋪為一維陣列
    return None

def load_dataset(dataset_dir):
    """ 載入數據集，並使用 MediaPipe 提取手部特徵點 """
    data = []
    labels = []
    class_names = sorted(os.listdir(dataset_dir))

    for label, class_name in enumerate(class_names):
        class_dir = os.path.join(dataset_dir, class_name)
        if not os.path.isdir(class_dir):
            continue
        
        for img_file in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_file)
            landmarks = extract_landmarks(img_path)
            if landmarks is not None:
                data.append(landmarks)
                labels.append(label)
    
    return np.array(data), np.array(labels), class_names

# 載入數據集
print("Loading dataset...")
data, labels, class_names = load_dataset(DATASET_DIR)
print(f"Loaded {len(data)} samples.")

# 保存標籤到文件
with open(LABELS_PATH, "w") as f:
    for class_name in class_names:
        f.write(class_name + "\n")
print(f"Labels saved to {LABELS_PATH}")

# 分割數據集
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# One-hot 編碼標籤
y_train = to_categorical(y_train, num_classes=len(class_names))
y_val = to_categorical(y_val, num_classes=len(class_names))

# ==============================
# Step 2: Build PointNet Model
# ==============================
def create_pointnet_model(input_shape, num_classes):
    """ PointNet 類神經網絡模型 """
    model = Sequential([
        Dense(128, activation='relu', input_shape=input_shape),
        Dropout(0.3),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0005),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# 創建模型
input_shape = (data.shape[1],)  # 每個樣本的特徵長度
num_classes = len(class_names)
model = create_pointnet_model(input_shape, num_classes)
model.summary()

# ==============================
# Step 3: Train the Model
# ==============================
print("Training model...")
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=64
)

# ==============================
# Step 4: Save Model and Labels
# ==============================
print("Saving model and labels...")
model.save(MODEL_PATH)
print(f"Model saved to {MODEL_PATH}")

Loading dataset...
Loaded 166015 samples.
Labels saved to C:\Users\jk121\Documents\Code\LargeData\models\labels.txt
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               8192      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 26)                3354      
      

c:\Users\jk121\.conda\envs\tensorflow-env\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
